## Data processing

Steps
1. Load cvs files
2. Merge files
3. correct data types
4. inputate NA
5. Feature engeneering


Import libraries

In [1]:
import pandas as pd
import datetime

1. Load cvs files

In [2]:
# Load a CSV files
df_kiwo = pd.read_csv(r'C:\Users\giuli\Documents\Open_Campus\bakery_prediction\0_DataPreparation\Raw\kiwo.csv')
df_daten = pd.read_csv(r'C:\Users\giuli\Documents\Open_Campus\bakery_prediction\0_DataPreparation\Raw\umsatzdaten_gekuerzt.csv')
df_wetter = pd.read_csv(r'C:\Users\giuli\Documents\Open_Campus\bakery_prediction\0_DataPreparation\Raw\wetter.csv')


2. Merge files

In [3]:
# merge dataframes on a common column
df_datenwetter = pd.merge(df_daten, df_wetter, on='Datum', how='left')

df_merged = pd.merge(df_datenwetter, df_kiwo, on='Datum', how='left')

In [4]:
# Display the first 5 rows
print(df_merged.head())

        id       Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0  1307011  2013-07-01            1  148.828353         6.0     17.8375   
1  1307021  2013-07-02            1  159.793757         3.0     17.3125   
2  1307031  2013-07-03            1  111.885594         7.0     21.0750   
3  1307041  2013-07-04            1  168.864941         7.0     18.8500   
4  1307051  2013-07-05            1  171.280754         5.0     19.9750   

   Windgeschwindigkeit  Wettercode  KielerWoche  
0                 15.0        20.0          NaN  
1                 10.0         NaN          NaN  
2                  6.0        61.0          NaN  
3                  7.0        20.0          NaN  
4                 12.0         NaN          NaN  


3. correct data types

In [8]:
#descriptive statistics
print(df_merged.describe())

#data types and non-null counts
print(df_merged.info())

#missing values in each column
#print(df_merged.isnull().sum())

                 id                          Datum  Warengruppe       Umsatz  \
count  9.334000e+03                           9334  9334.000000  9334.000000   
mean   1.559311e+06  2016-01-13 23:56:27.100921344     3.088172   206.749044   
min    1.307011e+06            2013-07-01 00:00:00     1.000000     7.051201   
25%    1.410123e+06            2014-10-12 00:00:00     2.000000    96.897441   
50%    1.601102e+06            2016-01-10 00:00:00     3.000000   161.900831   
75%    1.704223e+06            2017-04-22 00:00:00     4.000000   280.644663   
max    1.807315e+06            2018-07-31 00:00:00     6.000000  1879.461831   
std    1.512503e+05                            NaN     1.489002   144.545189   

        Bewoelkung   Temperatur  Windgeschwindigkeit   Wettercode  KielerWoche  
count  9264.000000  9318.000000          9318.000000  7009.000000        223.0  
mean      4.720747    12.028483            10.974780    36.900128          1.0  
min       0.000000    -8.475000     

In [6]:
#convert 'Datum' column to datetime
df_merged['Datum'] = pd.to_datetime(df_merged['Datum'], errors='coerce').dt.normalize()


In [9]:
# set warengruppe, bewoelkung, windgeschwindigkeit, wetter code and kieler woche as integer
df_merged['Warengruppe'] = df_merged['Warengruppe'].astype('int64')
df_merged['Bewoelkung'] = df_merged['Bewoelkung'].astype('Int64')
df_merged['Windgeschwindigkeit'] = df_merged['Windgeschwindigkeit'].astype('Int64')
df_merged['Wettercode'] = df_merged['Wettercode'].astype('Int64')
df_merged['KielerWoche'] = df_merged['KielerWoche'].astype('Int64')

Save to csv// import from csv

In [10]:
#save merged dataframe to a new CSV file
df_merged.to_csv(r'C:\Users\giuli\Documents\Open_Campus\bakery_prediction\0_DataPreparation\Processed\merged_data.csv', index=False)

In [56]:
df_merged = pd.read_csv(r'C:\Users\giuli\Documents\Open_Campus\bakery_prediction\0_DataPreparation\Processed\merged_data.csv')

4. inputate NA

In [11]:
df_inputated = df_merged.copy()

In [27]:
#missing values in each column
print(df_inputated.isnull().sum())

id                      0
Datum                   0
Warengruppe             0
Umsatz                  0
Bewoelkung             24
Temperatur             24
Windgeschwindigkeit    24
Wettercode              0
KielerWoche             0
Woche                   0
Monat                   0
dtype: int64


Categorical Wettercode per week mode

In [13]:
# derive week and month for df_inputated
df_inputated['Woche'] = pd.to_datetime(df_inputated['Datum']).dt.isocalendar().week
df_inputated['Monat'] = pd.to_datetime(df_inputated['Datum']).dt.month

# precompute weekly and monthly modes (ignore NaNs when computing mode)
weekly_mode_map = df_inputated.groupby('Woche')['Wettercode'].agg(
    lambda s: s.dropna().mode().iloc[0] if not s.dropna().mode().empty else pd.NA
).to_dict()

monthly_mode_map = df_inputated.groupby('Monat')['Wettercode'].agg(
    lambda s: s.dropna().mode().iloc[0] if not s.dropna().mode().empty else pd.NA
).to_dict()

# fill missing Wettercode: use weekly mode, if not available fallback to monthly mode
def fill_wettercode_row(row):
    if pd.isna(row['Wettercode']):
        w_mode = weekly_mode_map.get(row['Woche'], pd.NA)
        if not pd.isna(w_mode):
            return w_mode
        return monthly_mode_map.get(row['Monat'], pd.NA)
    return row['Wettercode']

df_inputated['Wettercode'] = df_inputated.apply(fill_wettercode_row, axis=1)

set not kiwo to 0 

In [15]:
#fix kieler woche missing values by filling with 0
df_inputated['KielerWoche'].fillna(0, inplace=True)

#set 'KielerWoche' as integer (nullable int64 to allow current NaNs)
df_inputated['KielerWoche'] = df_inputated['KielerWoche'].astype('int64')

C:\Users\giuli\AppData\Local\Temp\ipykernel_12080\3859340180.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_inputated['KielerWoche'].fillna(0, inplace=True)


In [17]:
#what days are missing Bewoelkung, Temperatur and/or     Windgeschwindigkeit
missing_weather = df_inputated[df_inputated[['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']].isnull().any(axis=1)]
print(missing_weather[['Datum', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']])

          Datum  Bewoelkung  Temperatur  Windgeschwindigkeit
1236 2016-12-11        <NA>         NaN                 <NA>
1526 2017-10-04        <NA>         NaN                 <NA>
1527 2017-10-05        <NA>         NaN                 <NA>
1551 2017-10-30        <NA>      8.1000                   11
1552 2017-11-01        <NA>     12.9750                   12
...         ...         ...         ...                  ...
9282 2017-11-04        <NA>     10.2625                    5
9283 2017-11-05        <NA>      9.8000                    8
9284 2017-11-06        <NA>      8.2125                    7
9285 2017-11-07        <NA>      6.7875                    5
9286 2017-11-08        <NA>      8.6250                    9

[70 rows x 4 columns]


fill Bewoelkung, Temperatur, Windgeschwindigkeit to mean of day before and after

In [28]:
#fill missing weather data with the mean of day before and after
df_inputated['Bewoelkung'] = df_inputated['Bewoelkung'].interpolate()
df_inputated['Temperatur'] = df_inputated['Temperatur'].interpolate()
df_inputated['Windgeschwindigkeit'] = df_inputated['Windgeschwindigkeit'].interpolate()



In [29]:
# set warengruppe, bewoelkung, windgeschwindigkeit, wetter code and kieler woche as integer
df_inputated['Bewoelkung'] = df_inputated['Bewoelkung'].astype('int64')
df_inputated['Windgeschwindigkeit'] = df_inputated['Windgeschwindigkeit'].astype('int64')

In [ ]:
#median values before and after imputation from 2016-12-11 to 2017-11-08
#filter date range
df_merged = df_merged[(df_merged['Datum'] >= '2016-12-11') & (df_merged['Datum'] <= '2017-11-08')]
df_inputated = df_inputated[(df_inputated['Datum'] >= '2016-12-11') & (df_inputated['Datum'] <= '2017-11-08')]  
print("Mean Bewoelkung before imputation:", df_merged['Bewoelkung'].mean())
print("Mean Bewoelkung after imputation:", df_inputated['Bewoelkung'].mean())
print("Mean Temperatur before imputation:", df_merged['Temperatur'].mean())
print("Mean Temperatur after imputation:", df_inputated['Temperatur'].mean())
print("Mean Windgeschwindigkeit before imputation:", df_merged['Windgeschwindigkeit'].mean())
print("Mean Windgeschwindigkeit after imputation:", df_inputated['Windgeschwindigkeit'].mean())

Mean Bewoelkung before imputation: 4.9167725540025415
Mean Bewoelkung after imputation: 4.867396593673966
Mean Temperatur before imputation: 12.191302867936118
Mean Temperatur after imputation: 12.184498825425791
Mean Windgeschwindigkeit before imputation: 10.947174447174447
Mean Windgeschwindigkeit after imputation: 10.945255474452555


In [31]:
# check if there are discrepancies for the same date for the columns 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit'
#for 'Warengruppe' 6 the temperature is different for the same date as the other 'Warengruppe'
discrepancies = df_inputated.groupby('Datum').agg({
    'Bewoelkung': pd.Series.nunique,
    'Temperatur': pd.Series.nunique,
    'Windgeschwindigkeit': pd.Series.nunique
}).reset_index() 
discrepancies = discrepancies[(discrepancies['Bewoelkung'] > 1) | (discrepancies['Temperatur'] > 1) | (discrepancies['Windgeschwindigkeit'] > 1)]
print("Discrepancies in weather data for the same date:")
print(discrepancies)

Discrepancies in weather data for the same date:
          Datum  Bewoelkung  Temperatur  Windgeschwindigkeit
1551 2017-10-30           2           2                    2
1552 2017-11-01           2           2                    2


In [23]:
#use the data from 'Warengruppe' 1 for the weather columns where discrepancies are found
for date in discrepancies['Datum']:
    weather_values = df_inputated[df_inputated['Datum'] == date][['Warengruppe', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']]
    reference_values = weather_values[weather_values['Warengruppe'] == 1][['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']].iloc[0]
    df_inputated.loc[df_inputated['Datum'] == date, ['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']] = reference_values 
    

Save to csv// import from csv

In [30]:
#save inputated dataframe to a new CSV file

df_inputated.to_csv(r'C:\Users\giuli\Documents\Open_Campus\bakery_prediction\0_DataPreparation\Processed\inputated_data.csv', index=False)

In [ ]:
df_inputated = pd.read_csv(r'C:\Users\giuli\Documents\Open_Campus\bakery_prediction\0_DataPreparation\Processed\inputated_data.csv')

5. Feature engeneering

In [71]:
df_featured = df_inputated.copy() 

insert day of the week

In [72]:
#insert new column day of week as integer (1=Monday,...,7=Sunday)   
df_featured['Wochentag'] = pd.to_datetime(df_featured['Datum']).dt.dayofweek + 1
print(df_featured[['Datum', 'Wochentag']].head())


          Datum  Wochentag
1236 2016-12-11          7
1237 2016-12-12          1
1238 2016-12-13          2
1239 2016-12-14          3
1240 2016-12-15          4


insert holiday

In [80]:
#insert new column holiday from 2013-07-01 to 2018-07-31 
holidays = [[
    '2013-10-03', '2013-12-25', '2013-12-26',
    '2014-01-01', '2014-04-18', '2014-04-21',
    '2014-05-01', '2014-05-29', '2014-06-09',
    '2014-10-03', '2014-12-25', '2014-12-26',
    '2015-01-01', '2015-04-03', '2015-04-06',
    '2015-05-01', '2015-05-14', '2015-05-25',
    '2015-10-03', '2015-12-25', '2015-12-26',
    '2016-01-01', '2016-03-25', '2016-03-28',
    '2016-05-01', '2016-05-05', '2016-05-16',
    '2016-10-03', '2016-12-25', '2016-12-26',
    '2017-01-01', '2017-04-14', '2017-04-17',
    '2017-05-01', '2017-05-25', '2017-06-05',
    '2017-10-03', '2017-10-31', '2017-12-25', '2017-12-26',
    '2018-01-01', '2018-03-30', '2018-04-02',
    '2018-05-01', '2018-05-10', '2018-05-21'
]]
df_featured['Feiertag'] = df_featured['Datum'].isin(holidays)
#convert boolean to integer (0 not holiday, 1 holiday)
df_featured['Feiertag'] = df_featured['Feiertag'].astype(int)

print(df_featured[['Datum', 'Feiertag']].head())

          Datum  Feiertag
1236 2016-12-11         0
1237 2016-12-12         0
1238 2016-12-13         0
1239 2016-12-14         0
1240 2016-12-15         0


set season from day and month

In [77]:
import datetime

# create column season like this: Winter from 22 December to 20 march, Spring from 21 March to 21 june, Summer from 22 June to 21 september, Autumn from 22 September to 21 december
def get_season(date_or_month):
    # If a date-like is passed (string/Timestamp/date), use exact day-month boundaries
    if isinstance(date_or_month, (str, pd.Timestamp, datetime.date)):
        d = pd.to_datetime(date_or_month)
        m, day = d.month, d.day
        if (m == 12 and day >= 22) or m in (1, 2) or (m == 3 and day <= 20):
            return 1 #'Winter'
        if (m == 3 and day >= 21) or m in (4, 5) or (m == 6 and day <= 21):
            return 2 #'Spring'
        if (m == 6 and day >= 22) or m in (7, 8) or (m == 9 and day <= 21):
            return 3 #'Summer'
        return 4 #'Autumn'
    # If an integer month is passed, fall back to month-based mapping
    m = int(date_or_month)
    if m in (12, 1, 2):
        return 1 #'Winter'
    if m in (3, 4, 5):
        return 2 #'Spring'
    if m in (6, 7, 8):
        return 3 #'Summer'
    return 4 #'Autumn'

df_featured['Monat'] = pd.to_datetime(df_featured['Datum']).dt.month
# Derive season based on the full date (day-month boundaries are respected)
df_featured['Jahreszeit'] = pd.to_datetime(df_featured['Datum']).apply(get_season)
print(df_featured[['Datum', 'Monat', 'Jahreszeit']].head())


          Datum  Monat  Jahreszeit
1236 2016-12-11     12           4
1237 2016-12-12     12           4
1238 2016-12-13     12           4
1239 2016-12-14     12           4
1240 2016-12-15     12           4


In [ ]:
# insert new column for school holidays (ferien)


In [82]:
#save featured dataframe to a new CSV file
df_featured.to_csv(r'C:\Users\giuli\Documents\Open_Campus\bakery_prediction\0_DataPreparation\Processed\featured_data.csv', index=False)